In [ ]:
#Lets upload the data into google collab
from google.colab import files
uploaded = files.upload()

Saving Trimet.csv to Trimet.csv


In [ ]:
#import required Libraries
import pandas as pd
from datetime import datetime, timedelta


In [ ]:
data  = pd.read_csv('Trimet.csv')
print(data.head(5))

   EVENT_NO_TRIP  EVENT_NO_STOP            OPD_DATE  VEHICLE_ID  METERS  \
0      259172515      259172517  15FEB2023:00:00:00        4223      40   
1      259172515      259172517  15FEB2023:00:00:00        4223      48   
2      259172515      259172517  15FEB2023:00:00:00        4223      57   
3      259172515      259172517  15FEB2023:00:00:00        4223      73   
4      259172515      259172517  15FEB2023:00:00:00        4223     112   

   ACT_TIME  GPS_LONGITUDE  GPS_LATITUDE  GPS_SATELLITES  GPS_HDOP  
0     20469    -122.648137     45.493082              12       0.7  
1     20474    -122.648240     45.493070              12       0.8  
2     20479    -122.648352     45.493123              12       0.8  
3     20484    -122.648385     45.493262              12       0.7  
4     20489    -122.648347     45.493582              12       0.8  


In [ ]:
#Lets drop unnecessary columns

new_data = data.drop(columns=['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'])
print(new_data.head())


   EVENT_NO_TRIP            OPD_DATE  VEHICLE_ID  METERS  ACT_TIME  \
0      259172515  15FEB2023:00:00:00        4223      40     20469   
1      259172515  15FEB2023:00:00:00        4223      48     20474   
2      259172515  15FEB2023:00:00:00        4223      57     20479   
3      259172515  15FEB2023:00:00:00        4223      73     20484   
4      259172515  15FEB2023:00:00:00        4223     112     20489   

   GPS_LONGITUDE  GPS_LATITUDE  
0    -122.648137     45.493082  
1    -122.648240     45.493070  
2    -122.648352     45.493123  
3    -122.648385     45.493262  
4    -122.648347     45.493582  


In [ ]:
#function to determine which columns to load
def columns(cols):
    columns = ['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP']
    return [col for col in cols if col not in columns]

filtered_data = pd.read_csv('Trimet.csv', usecols=lambda column : column not in ['EVENT_NO_STOP', 'GPS_SATELLITES', 'GPS_HDOP'])

print(filtered_data.head())

   EVENT_NO_TRIP            OPD_DATE  VEHICLE_ID  METERS  ACT_TIME  \
0      259172515  15FEB2023:00:00:00        4223      40     20469   
1      259172515  15FEB2023:00:00:00        4223      48     20474   
2      259172515  15FEB2023:00:00:00        4223      57     20479   
3      259172515  15FEB2023:00:00:00        4223      73     20484   
4      259172515  15FEB2023:00:00:00        4223     112     20489   

   GPS_LONGITUDE  GPS_LATITUDE  
0    -122.648137     45.493082  
1    -122.648240     45.493070  
2    -122.648352     45.493123  
3    -122.648385     45.493262  
4    -122.648347     45.493582  


In [ ]:

#Creating a single timestamp

def timestamp(row):

    opd_date = datetime.strptime(row['OPD_DATE'], '%d%b%Y:%H:%M:%S')

    act_time = timedelta(seconds=int(row['ACT_TIME']))

    return opd_date + act_time

# Apply the function to each row to create the new TIMESTAMP column in our data

data['TIMESTAMP'] = data.apply(timestamp, axis=1)

print(data['TIMESTAMP'].head(5))

0   2023-02-15 05:41:09
1   2023-02-15 05:41:14
2   2023-02-15 05:41:19
3   2023-02-15 05:41:24
4   2023-02-15 05:41:29
Name: TIMESTAMP, dtype: datetime64[ns]


In [ ]:
#Dropped the unnecessary columns
data = data.drop(columns=['OPD_DATE', 'ACT_TIME'])

In [18]:
# Calculating the differences in METERS and TIMESTAMP

def calculate_speed(data):
  data['temp_METERS'] = data['METERS'].diff()
  data['temp_TIMESTAMP'] = data['TIMESTAMP'].diff().dt.total_seconds()
  data['SPEED'] = data['temp_METERS'] / data[data['temp_TIMESTAMP'] != 0]['temp_TIMESTAMP']
  return data.drop(columns=['temp_METERS', 'temp_TIMESTAMP'])

data = calculate_speed(data.copy())

# Speed Insights
print(data['SPEED'].describe())

count    160.000000
mean       7.227206
std        4.420604
min        0.000000
25%        3.800000
50%        6.400000
75%       10.850000
max       17.400000
Name: SPEED, dtype: float64
